## Library

In [1]:
import platform
import os
import random

from sklearn.metrics import f1_score, classification_report
import efficientnet
import efficientnet.tfkeras as efn
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import sklearn


In [2]:
SEED = 42

os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)


In [3]:
print('Python version:', platform.python_version())
print('Tensorflow Version:', tf.__version__)
print('Tensorflow Addons Version:', tfa.__version__)
print('Efficientnet Version:', efficientnet.__version__)
print('Numpy Version:', np.__version__)


Python version: 3.8.3
Tensorflow Version: 2.2.0
Tensorflow Addons Version: 0.10.0
Efficientnet Version: 1.1.0
Numpy Version: 1.18.5


## Dataset

In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()


## Configure model

In [5]:
efn0 = efn.EfficientNetB0(
    input_shape=(32, 32, 3), include_top=False,
    weights='noisy-student', pooling='max',
)
model = tf.keras.Sequential([
    efn0,
    Dense(10, activation='softmax')
])
model.compile(
    optimizer=tfa.optimizers.RectifiedAdam(
        lr=0.005,
        total_steps=50,
        warmup_proportion=0.1,
        min_lr=0.0005,
    ),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b0 (Model)      (None, 1280)              4049564   
_________________________________________________________________
dense (Dense)                (None, 10)                12810     
Total params: 4,062,374
Trainable params: 4,020,358
Non-trainable params: 42,016
_________________________________________________________________


## Train model

In [6]:
model.fit(
    x_train, y_train,
    batch_size=500, epochs=50, verbose=1
)


Epoch 1/50
100/100 [==============================] - 19s 185ms/step - loss: 1.8861 - sparse_categorical_accuracy: 0.3422
Epoch 2/50
100/100 [==============================] - 19s 185ms/step - loss: 1.3120 - sparse_categorical_accuracy: 0.5429
Epoch 3/50
100/100 [==============================] - 18s 183ms/step - loss: 1.0185 - sparse_categorical_accuracy: 0.6420
Epoch 4/50
100/100 [==============================] - 18s 184ms/step - loss: 0.8267 - sparse_categorical_accuracy: 0.7108
Epoch 5/50
100/100 [==============================] - 19s 186ms/step - loss: 0.6838 - sparse_categorical_accuracy: 0.7578
Epoch 6/50
100/100 [==============================] - 19s 186ms/step - loss: 0.5767 - sparse_categorical_accuracy: 0.7973
Epoch 7/50
100/100 [==============================] - 19s 185ms/step - loss: 0.4832 - sparse_categorical_accuracy: 0.8324
Epoch 8/50
100/100 [==============================] - 19s 186ms/step - loss: 0.3956 - sparse_categorical_accuracy: 0.8603
Epoch 9/50
100/100 [====

## Test model

In [7]:
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=-1)


In [8]:
f1 = f1_score(y_test, y_pred, average='weighted')

print('Weighted F1 Score:', f1)


Weighted F1 Score: 0.8101679207753008


In [9]:
print('Classification Report:')
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.88      0.84      1000
           1       0.93      0.83      0.88      1000
           2       0.83      0.70      0.76      1000
           3       0.67      0.60      0.63      1000
           4       0.73      0.85      0.78      1000
           5       0.66      0.79      0.72      1000
           6       0.87      0.84      0.86      1000
           7       0.89      0.83      0.86      1000
           8       0.92      0.89      0.90      1000
           9       0.85      0.90      0.87      1000

    accuracy                           0.81     10000
   macro avg       0.82      0.81      0.81     10000
weighted avg       0.82      0.81      0.81     10000

